In [54]:
from uplink import Consumer, get, post, headers, Path, Query, Field

In [107]:
class USASpending(Consumer):
    """A Python Client for the USASpending API."""

    @get("/api/v2/awards/last_updated/")
    def get_awards_last_updated_date(self):
        """Returns the last-updated date for the Award data."""
        
    @get("/api/v2/federal_accounts/{federal_account_id}/available_object_classes")
    def get_object_class_by_federal_account(self, federal_account_id):
        """Returns minor object classes rolled up under major classes, filtered by federal account."""
        
    @get("/api/v2/federal_accounts/{federal_account_id}/fiscal_year_snapshot/{fiscal_year}/")
    def get_fiscal_year_snapshot(self, federal_account_id, fiscal_year=2018):
        """Returns budget information for a federal account. 
        If no fiscal year is used, the federal account's most recent fiscal year is used as the default."""
        
    @post("/api/v2/federal_accounts/")
    def post_federal_accounts(self,
                              page: Field=1,
                              limit: Field=10
                              ):
        """Returns a list of federal accounts."""

In [108]:
usaspending = USASpending(base_url="https://api.usaspending.gov/")

In [6]:
awards_updated = usaspending.get_awards_last_updated_date()
awards_updated.json()

{'last_updated': '12/10/2018'}

In [52]:
# Get list of all federal accounts
def get_all_federal_accounts():
    results = []
    has_next = True
    page = 1

    while has_next:
        federal_accounts = usaspending.post_federal_accounts(page=page, limit=100)
        data = federal_accounts.json()
        # print(f'Adding page {page} of {int(data.get("count") / data.get("limit") + 0.99)}')
        results.extend(data.get('results'))
        has_next = data.get('hasNext')
        page += 1
              
    return results

In [55]:
fed_accounts = get_all_federal_accounts()

In [96]:
fed_accounts[0]

{'managing_agency_acronym': 'CFTC',
 'account_number': '339-1402',
 'account_name': 'Expenses, Commodity Futures Trading Commission, Unliquidating Deficiency',
 'account_id': 6102,
 'managing_agency': 'Commodity Futures Trading Commission',
 'agency_identifier': '339',
 'budgetary_resources': -149391065.92}

In [97]:
[(agency['managing_agency'], agency['account_id']) for agency in fed_accounts][:10]

[('Commodity Futures Trading Commission', 6102),
 ('Department of State', 4254),
 ('Executive Office of the President', 3354),
 ('Department of Transportation', 4990),
 ('Department of Transportation', 4988),
 ('Department of Transportation', 4979),
 ('Department of Transportation', 4986),
 ('Department of Transportation', 4975),
 ('Department of Transportation', 4981),
 ('Department of Transportation', 4985)]

In [98]:
def lookup_agency_by_name(name, fed_accounts):
    '''Returns a list of agencies that contain name provided'''
    return [(agency['managing_agency'], agency['account_name'], agency['account_id'])
            for agency in fed_accounts
            if name.lower() in str(agency['managing_agency']).lower()]

In [99]:
lookup_agency_by_name('air force', fed_accounts)[:5]

[('Department of the Air Force', 'Air Force Cadet Fund, Air Force', 4843),
 ('Department of the Air Force',
  'Military Construction - Recovery Act, Air National Guard',
  4817),
 ('Department of the Air Force',
  'Family Housing Construction - Recovery Act, Air Force',
  4792),
 ('Department of the Air Force',
  'Tanker Replacement Transfer Fund, Air Force',
  4803),
 ('Department of the Air Force',
  'Military Construction - Recovery Act, Air Force',
  4806)]

In [101]:
af_cadet_fund = '4843'
af_cadet_obj_class = usaspending.get_object_class_by_federal_account(af_cadet_fund)
af_cadet_obj_class.json()['results']

[{'minor_object_class': [{'name': 'Refunds', 'id': '440'}],
  'name': 'Grants and fixed charges',
  'id': '40'},
 {'minor_object_class': [{'name': '', 'id': '000'}],
  'name': 'Unknown',
  'id': '00'}]

In [110]:
af_cadet_fy_snapshot = usaspending.get_fiscal_year_snapshot(af_cadet_fund, 2017)
af_cadet_fy_snapshot.json()

{'results': {'name': 'Air Force Cadet Fund, Air Force',
  'unobligated': 0.0,
  'other_budgetary_resources': 252678.96,
  'outlay': 0.0,
  'balance_brought_forward': 0.0,
  'budget_authority': 252678.96,
  'appropriations': 0.0,
  'obligated': 252678.96}}

In [111]:
af_cadet_fy_snapshot = usaspending.get_fiscal_year_snapshot(af_cadet_fund, 2018)
af_cadet_fy_snapshot.json()

{'results': {'name': 'Air Force Cadet Fund, Air Force',
  'unobligated': 0.0,
  'other_budgetary_resources': 0.0,
  'outlay': -431940.04,
  'balance_brought_forward': 0.0,
  'budget_authority': 0.0,
  'appropriations': 0.0,
  'obligated': 0.0}}